In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_excel('XXX')
project=data
data
        

In [ ]:
projects = ['p50_david_dietz_2020','p50_hao_chen_2020','p50_hao_chen_2020_rnaseq','p50_paul_meyer_2020',
            'u01_olivier_george_cocaine', 'u01_olivier_george_oxycodone', 'u01_olivier_george_scrub',
            'u01_peter_kalivas_italy', 'u01_peter_kalivas_us', 'u01_suzanne_mitchell', 'u01_tom_jhou' ]
def runQuery(query):
    connection = psycopg2.connect(database = "PalmerLab_Datasets",
                                  user = "postgres",
                                  password = "palmerlab-amapostgres",
                                  host = "palmerlab-main-database-c2021-08-02.c6sgfwysomht.us-west-2.rds.amazonaws.com",
                                  port = '5432')
    cursor = connection.cursor()
    if "SELECT" in query:
        table = pd.read_sql(query, con = connection)
    elif "UPDATE" in query:
        cursor.execute(query)
        print(query)
        connection.commit()
    elif "DELETE" in query:
        cursor.execute(query)
        print(query)
        connection.commit()
    else:
        return None
    if connection:
        cursor.close()
        connection.close()
        if "SELECT" in query:
            return table
        else:
            return
# #change project here
current_project = 'u01_peter_kalivas_italy.wfu_master'
wfu = runQuery("SELECT * FROM " + current_project)
wfu
# wfu_master['project_name'].value_counts()

# wfu_master['cohort'].value_counts().sort_index()


In [ ]:
def fixColumns(columns):
    for i in range(len(columns)):
        columns[i] = columns[i].lower().replace(" ", "")
        if "sir" in columns[i] or "father" in columns[i]:
            columns[i] = "sires"
        elif "dam" in columns[i] or "mother" in columns[i]:
            columns[i] = "dames"
        elif "animal" in columns[i] and "id" in columns[i]:
            columns[i] = "labanimalid"
        elif "access" in columns[i] and "id" in columns[i]:
            columns[i] = "accessid"
        elif "sex" in columns[i]:
            columns[i] = "sex"
        elif "rfid" in columns[i] or "ponder" in columns[i]:
            columns[i] = "rfid"
        elif ("d" in columns[i] and "o" in columns[i] and "b" in columns[i]) or "birth" in columns[i]:
            columns[i] = "dob"
        elif ("d" in columns[i] and "o" in columns[i] and "w" in columns[i]) or ("wean" in columns[i] and "date" in columns[i]):
            columns[i] = "dow"
        elif "ship" in columns[i] and "date" in columns[i]:
            columns[i] = "shipmentdate"
        elif "litter" in columns[i] and ("#" in columns[i] or "num" in columns[i]):
            columns[i] = "litternumber"
        elif "ship" in columns[i] and "box" in columns[i]:
            columns[i] = "shipmentbox"
        elif "comment" in columns[i] or "note" in columns[i]:
            columns[i] = "comments"
    return columns


In [ ]:
def checkDupCols(data):
    for col in data.columns:
        if ".1" in col:
            data[col] = data[col].astype(str)
            data[col.replace(".1", "")] = data[col.replace(".1", "")].astype(str)
            data[col] = [val.replace(".0", "") for val in data[col]]
            data[col.replace(".1", "")] = [val.replace(".0", "") for val in data[col].replace(".1", "")]
            if list(data[col] == data[col.replace(".1", "")]).count(True) != len(data[col]):
                print("Transponder ID Mismatch found")
                print(data[col])
                print(data[col.replace(".1", "")])
            del data[col]
    return data

In [ ]:
#Specific Errors

def date_error(data):
    #Meyer2020 Shipping Sheet #7
    a = (data['dob'] == '4/21/20221')
    data.loc[a,'dob'] = '4/21/2021'
    return data

def comment_error(data):
    #Meyer2020 Shipping Sheet #7
    #data["comments"] = "NA"
    if 'unnamed:18' in data.columns:
        data.loc[data['unnamed:18'] == 'DOUBLE PULLS','unnamed:18'] = np.nan
        data.loc[(data['rfid'] == 'AA1DCD6794'),'comments'] = "DOUBLE PULLS"
        data = data.drop(columns = ['unnamed:18'])
    return data
    
def column_error(data):
    #Meyer2020 Shipping sheet #7
    if '124rats' in data.columns:
        data = data.drop(columns=['124rats'])
    return data

def error_rows(data):
    #Meyer2020 Shipping sheet #7
    #Meyer2020 Shipping sheet #8
    if 'unnamed:5' in data.columns:
        print("Difference between unnamed:5 and rfid")
        display(data.loc[~(data['unnamed:5'] == data['rfid'])])
        print("unnamed:5 value counts")
        display(data['unnamed:5'].value_counts().to_string())
        print("Dropped unnamed:5")
        data = data.drop(columns = ['unnamed:5'])
    return data
        
def error_rfids(data):
    data['rfid'] = data['rfid'].str.upper()

In [ ]:
#calculate the weanage and shipment age
def addAge(data, col_name):
    data['dob'] = pd.to_datetime(data['dob'],errors='coerce')
    data['dow'] = pd.to_datetime(data['dow'],errors='coerce')
    data['shipmentdate'] = pd.to_datetime(data['shipmentdate'],errors='coerce')

    if "ship" in col_name:
        i_date = list(data.columns).index("shipmentdate")
    elif "wean" in col_name:
        i_date = list(data.columns).index("dow")
    else:
        print("Invalid col_name provided")
        return None
    values = []
    for index, row in data.iterrows():
        i_dob = list(data.columns).index("dob")
        # Want it to be datetime.date not datetime.datetime
        if not isinstance(data.iloc[index, i_dob], datetime.datetime):
            data.iloc[index, i_dob] = data.iloc[index, i_dob].date()
        if not isinstance(data.iloc[index, i_date], datetime.datetime):
            data.iloc[index, i_date] = data.iloc[index, i_date].date()
        values.append((data.iloc[index, i_date] - data.iloc[index, i_dob]).days)
    return values

# weanage=addAge(data, 'weanage')
# data['weanage']=weanage
# data

In [ ]:
#change transponder ID into rfid
def rfid_column():
    if "Transponder ID" in data.columns:
        i = list(data.columns).index("Transponder ID")
        for index, row in data.iterrows():
            data.iloc[index, i] = data.iloc[index, i].upper()
            if data.iloc[index, i][:2] != "AA":
                data.iloc[index, i] = "AA" + data.iloc[index, i]
        checkDupCols(data)
        data.columns = fixColumns(list(data.columns))
#rfid_column()

In [ ]:
# Check if data columns look correct
#print("Data columns:", list(data.columns))
#remove NA columns
def drop_na(data):
    list1 = data.columns
    #print(list1)
    data = data.dropna(axis=1, how='all')
    list2 = data.columns
    #print(list2)
    diff = list(set(list1).difference(list2))
    if len(diff) >= 1:
        print("\nData columns dropped due to all NA:", diff)
    return data


In [ ]:
def rfid_in_wfu(data):
    data["rfid"] = data["rfid"].astype(str)
    wfu_master_rfids = list(wfu_master["rfid"])
    data_rfids = list(data["rfid"])
    duplicate_rfids = list(set(wfu_master_rfids).intersection(set(data_rfids)))
    print(len(duplicate_rfids), "rfids already in WFU_master, out of", len(data_rfids))
#     print(duplicate_rfids)
    fix = list(set(data_rfids) - set(wfu_master_rfids))


In [ ]:
def new_columns(data):
    if True:
        # Making wfu_master and data have same columns
        cohort = "".join(filter(str.isdigit, project))
        if len(cohort) == 1:
            cohort = "0" + cohort
        data["cohort"] = "C" + cohort
        data["shipmentage"] = addAge(data, "shipmentage")
        data["weanage"] = addAge(data, "weanage")
        data["comments"] = "NA"
        data["resolution"] = "NA"
        #data = data[columns_wfu]
        for index, row in data.iterrows():
            for i in range(len(row)):
                if isinstance(data.iloc[index, i], datetime.date):
                    if isinstance(data.iloc[index, i], type(pd.NaT)):
                        data.iloc[index, i] = None
                    else:
                        data.iloc[index, i] = data.iloc[index, i].strftime("%Y/%m/%d")

# new_columns(data)


In [ ]:
def column_qc():
    print("\n\nColumn headers for: "+ current_project)
    for column in data.columns:
        print(column)
    common_headers = ['cohort','sires','dames','labanimalid','accessid','sex','rfid','dob','dow','shipmentdate',
                      'litternumber','littersize','coatcolor','earpunch','rack','shipmentbox','shipmentage','weanage',
                      'comments','resolution']
    #find any odd column headers
    for column in data.columns:
        if column not in common_headers:
            print("\nNon uniform column headers:")
            print(column)
    for column in common_headers:
        if column not in data.columns:
            print("\nMissing uniform column header:")
            print(column)


In [ ]:
# wfu_master
sheet="XXX"

In [ ]:

def cohorts(data):
    print("\nNumber of cohorts for " + sheet + ":")
    display(data['cohort'].value_counts().sort_index())
    print("\nCohort for " + current_project +", " " and the number of subjects:")
    display(wfu_master['cohort'].value_counts().sort_index())
    
# cohorts(data)

In [ ]:
#count the number of sires and dames in each cohort
#count how many of those sires and dames are unique meaning that multiple sires and dames had more than one child
#check how many parent pairs there are
#Check how many parent pairs are present in multiple cohorts
#check if there is an rfid labeled as both a sire and a dame
#check for missing parents
def qc_pedigree(data):
    sires = list(data["sires"])
    dames = list(data["dames"])
    
    data = data.sort_values(["cohort", "labanimalid"])

#     Printing number of mothers and fathers per cohort
    unique_sires = []
    unique_dames = []
    for cohort in np.unique(data["cohort"]):
        subset = data[data["cohort"] == cohort]
        sires = list(subset["sires"])
        dames = list(subset["dames"])
        unique_sires.append(len(set(sires)))
        unique_dames.append(len(set(dames)))
        print(cohort, "sires", "unique", len(set(sires)))
        print(cohort, "dames", "unique", len(set(dames)))


#     Checking sibling numbers
    parents_dict = {}
    parents_litter_birth = {}
    dob = []
    litter = []
    parents_df = pd.DataFrame()
    for index, row in data.iterrows():
        parents = (row["sires"], row["dames"])
        
        if parents in parents_dict.keys():
            parents_dict[parents].append(row["rfid"])
            parents_litter_birth[parents].append(row["dob"])
        else:
            dob.append(row['dob'])
            litter.append(row['littersize'])
            parents_dict[parents] = [row["rfid"]]
            parents_litter_birth[parents] = [row["dob"]]
            parents_df = parents_df.append({'sires':row["sires"],'dames':row["dames"]},ignore_index = True)
    print("\nparent pairs",len(parents_dict.keys()), '\n')
    siblings = []
    for parents in parents_dict.keys():
        #print(parents, len(parents_dict[parents]))#, parents_dict[parents])
        siblings.append(len(parents_dict[parents]))
        cohorts = []
        for rfid in parents_dict[parents]:
            subset = data[data["rfid"] == rfid]
            subset.index = range(subset.shape[0])
            i_cohort = list(subset.columns).index("cohort")
            cohorts.append(subset.iloc[0, i_cohort])
        cohorts = list(np.unique(cohorts))
        if len(cohorts) != 1:
            print("parent pair across multiple cohorts found", cohorts)
    parents_df['siblings'] = siblings
    parents_df['dob'] = dob
    parents_df['litter_size'] = litter
#     parents_df['removed_litter'] = parents_df['litter_size'] - parents_df['siblings']
    

        
    #Check if there are any matching ids between sires and dames
    overlap = set(dames).intersection(set(sires))
    if len(list(overlap)) > 0:
        print("\nError: listed as dame and sire", list(overlap))
    elif len(list(overlap)) == 0:
        print('\nNo rfids labeled as both a sire and a dame')
    mislabeled_df = pd.DataFrame()
    unique_sires = data.sires.unique()
    unique_dames = data.dames.unique()
    mislabeled = [x for x in unique_sires if x in unique_dames]
    if len(overlap) > 0:
        mislabeled_nan_sires = data[data['sires'].isnull()]
        mislabeled_nan_dames = data[data['dames'].isnull()]
        mislabeled_nan = pd.concat([mislabeled_nan_sires,mislabeled_nan_dames]).drop_duplicates().reset_index(drop=True)
        if len(mislabeled_nan.index) > 0:
            print('\nMissing Sires/Dames for ' + current_project)
            display(mislabeled_nan)
        for rfid in mislabeled:
            mislabeled_df.append(data.loc[data['sires'] == rfid])
            mislabeled_df.append(data.loc[data['dames'] == rfid])
        if len(mislabeled_df.dropna().index) > 0:
            print('Mislabeled Sires/Dames')
            display(mislabeled_df.dropna())
        

    df_missing_sires = data[data.sires == 'None']
    df_missing_dames = data[data.dames == 'None']
    df_missing = pd.concat([df_missing_sires,df_missing_dames]).drop_duplicates().reset_index(drop=True)
    if len(df_missing) > 0:
        print('\nMissing Parents:')
        display(df_missing)
    else:
        print('\nNo missing parents')

# qc_pedigree(data)


In [ ]:
def birth_range():
    date_min = min(data['dob'])
    date_max = max(data['dob'])
    date_range = date_max.date() - date_min.date()
    print('\nDOB range(days):', date_range.days)
    
# birth_range()
# min(data['dob'])
# max(data['dob'])

In [ ]:
def wean_range():
    date_min = min(data['dow'])
    date_max = max(data['dow'])
    date_range = date_max - date_min
    print('\nDOW range(days):', date_range.days)

In [ ]:
# Fixing coatcolor in sample_metadata
def coat_color(data):
    toFix = []
    metadata = data
    print(" Coat Colors:")
    display(data['coatcolor'].value_counts().to_frame())

    # metadata = select_wfu_master("u01_olivier_george_oxycodone")
    i_coatcolor = list(metadata.columns).index("coatcolor")
    for index, row in metadata.iterrows():
        coatcolor = data.iloc[index, i_coatcolor]
        if coatcolor == "BLK HOOD":
            coatcolor = "BLACKHOOD"
        elif coatcolor == "BRN HOOD":
            coatcolor = "BROWNHOOD"
        elif coatcolor == "BEN HOOD":
            coatcolor = "BROWNHOOD"
        elif coatcolor == "ALIBINO":
            coatcolor = "ALBINO"
#         coatcolor = coatcolor.replace(" ", "")
        if coatcolor != metadata.iloc[index, i_coatcolor]:
            toFix.append(index)
            metadata.iloc[index, i_coatcolor] = coatcolor
    metadata = metadata.iloc[toFix, :]
    metadata.index = range(metadata.shape[0])
    print("\nAfter Coat Color Normalization:")
    display(data['coatcolor'].value_counts().to_frame())

coat_color(data) 
    if len(metadata.index) > 0:
        print("Need to normalize following rows for coat color:")
        display(metadata)
wfu_master

In [ ]:
def qc_rfid(rfid, prefix, length):
    # Checking if prefix is present
    if rfid[:len(prefix)] != prefix:
        return False
    # Checking is rfid is correct length
    elif len(rfid) != length:
        return False
    return True
def check_rfid():
    #Check to make sure rfid for the project contains the correct naming convention
    project_metadata = pd.read_csv("project_metadata - project_metadata (1).csv", dtype = str)
    i_convention = list(project_metadata.columns).index("rfid_convention")
    subset = project_metadata[project_metadata["project_name"] == 'u01_peter_kalivas_italy']
    subset.index = range(subset.shape[0])
    convention_list = subset.iloc[0, i_convention]
    convention_list = convention_list.split(";")
    for i in range(len(convention_list)):
        convention_list[i] = convention_list[i].replace("(", "").replace(")", "").split(",")
    for rfid in data["rfid"]:#data needs to be combination of all projec
        passed = False
        for convention in convention_list:
            prefix = convention[0]
            length = int(convention[1])
            passed = passed + qc_rfid(rfid, prefix, length)
        if not passed:
            print('RFID errors:')
            print(rfid)#, project)
            display(data.loc[data['rfid']== rfid])
         
        
     


    
# check_rfid()




In [ ]:
# data['siblings'] = np.nan
def siblings(data):
    data['siblings'] = np.nan 
    df = data.groupby(['dob','sires','dames'])['siblings'].count().reset_index()
    for index,row in df.iterrows():
        data.loc[(data["dob"] == row['dob']) & (data["sires"] == row['sires']) & (data['dames'] == row['dames']), "siblings"] = row['siblings']
    print("\nSibling value counts:")
    print("\n", data.siblings.value_counts().to_string())
    for index,row in data.iterrows():
        if row['siblings'] > 2:
            print("More than 2 siblings:\n")
            print(data.loc[[index]])
    return data

# siblings(data)




In [ ]:
def graph_columns():
    graph_columns = ['sex','dob','dow','shipmentage','coatcolor','weanage']#,'littersize','litternumber']
    for col in graph_columns:
        plt.title(col)
        plt.xlabel('value')
        plt.ylabel('count')
        plt.xticks(rotation = 40)
        plt.hist(data[col].dropna())
        plt.show() 

# graph_columns()

In [ ]:
# data=df3
wfu_master=wfu
#check duplicate columns
checkDupCols(data)
#fix column names
data.columns = fixColumns(list(data.columns))
#RFIDs already in wfu
rfid_in_wfu(data)
#correct specific errors involving dates
data = date_error(data)
#add column headers to make uniform with wfu
new_columns(data)
#correct specific errors involving comments
data = comment_error(data)
#correct specific errors involving columns
data = column_error(data)
#Drop columns with all NA values
data = drop_na(data)
#column information
column_qc()
#rows with notable errors
data = error_rows(data)
#cohort information
cohorts(data)
#calculate pedigree related information
qc_pedigree(data)
#caluclate the range of birth dates
birth_range()
#calculate the range of wean dates
wean_range()
#qc coat color
coat_color(data)
#error fixing for rfids
error_rfids(data)
#find inccorrect rfids
# check_rfid()
#sibling count
data = siblings(data)
#plot relevant graphs
graph_columns()

In [ ]:
## impliment any changes needed
data.columns
# data.drop(columns=['siblings', 'resolution'],inplace=True)
# data['comments']='Wrong RFIDS convention used by WFU'
data

In [ ]:
# to fix time stamp issue
# data['dob']=data['dob'].astype(str)
# data['dow']=data['dow'].astype(str)
# data['shipmentdate']=data['shipmentdate'].astype(str)
# data.drop(columns=['siblings', 'resolution'],inplace=True)
data

In [ ]:

def insertQuery(query, data):
    connection = psycopg2.connect(database = "PalmerLab_Datasets",
                                  user = "postgres",
                                  password = "palmerlab-amapostgres",
                                  host = "palmerlab-main-database-c2021-08-02.c6sgfwysomht.us-west-2.rds.amazonaws.com",
                                  port = '5432')
    cursor = connection.cursor()
    if "INSERT" in query:
        cols = ",".join([str(i) for i in data.columns.tolist()])
        for i, row in data.iloc[0:1, :].iterrows():
            query = query + " (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(query, tuple(row))
            print(list(row))
            connection.commit()
    else:
        return None
    if(connection):
        cursor.close()
        connection.close()
        return



# test=test.drop(columns=['cohort'])


# 9:21
project = "XXX"
query = "INSERT INTO " + project + "XXX"
data = data ## what does this mean 
data.index = range(data.shape[0])
print(list(data.columns))
for index, row in data.iterrows():
    print(index)
    print(list(row))
    insertQuery(query, data.iloc[index:index+1, :])
    
